In [1]:
import nltk
import numpy as np
import pandas as pd
from pandas import DataFrame
from keras.models import Model, Sequential, load_model
from keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.core import Activation, Dropout, Dense, Flatten
from keras.layers import TimeDistributed, Bidirectional, InputLayer, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import TensorBoard
from keras.metrics import categorical_accuracy
from IPython.display import clear_output
from more_itertools import flatten, intersperse
import random

import os
import urllib.request

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2
from batcher import batch_from_generator
from train_data import load_conll2003, create_conll_encoded_shifted_generator
from mappings import get_all_mappings, gen_input_feature_to_class_map, gen_input_feature_to_int_map
from corpus import corpus_training_data_generator, create_all_corpus_train_pipeline, pad, encode_each_sentence
from model import create_model, compile_model

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Todd\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Todd\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Todd\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [3]:
DROPOUT = 0.2
TIME_SLICE_SIZE = 50
BATCH_SIZE = 512
SAMPLING_RATE = 1
OUTPUT_CLASSES = 4
PADDING = 0
UNKNOWN = 1
NUM_OF_UNITS = 100
WORDS_PER_BATCH = 100
EPOCHS=1
MODEL_SAVE_PATH = 'tc_model.h5'
LSTM_MODEL_SAVE_PATH = 'lstm_model.h5'

In [4]:
mapping, reverse_mapping, lower_mapping, lower_reverse_mapping = get_all_mappings()
input_feature_to_class_map = gen_input_feature_to_class_map()
input_feature_to_int_map = gen_input_feature_to_int_map()

In [5]:
# def create_model(shape, classes):
#     # First layer inputs must be 3D
#     # with shape (samples, timesteps, features)
#     model = Sequential()
#     model.add(InputLayer(input_shape=shape))
#     model.add(Bidirectional(LSTM(NUM_OF_UNITS, return_sequences=True, dropout=DROPOUT, recurrent_dropout=DROPOUT)))
#     model.add(TimeDistributed(Dense(1000)))
#     model.add(TimeDistributed(Dense(classes)))
#     model.add(Activation('softmax'))
#     return model

In [12]:
model = create_model(1, 4, NUM_OF_UNITS, DROPOUT)

In [13]:
model = compile_model(model)

In [8]:
if os.path.isfile(MODEL_SAVE_PATH): 
    model.load_weights(MODEL_SAVE_PATH)

In [9]:
tensor_board = TensorBoard(batch_size=BATCH_SIZE, write_graph=False, )

In [10]:

X, Y = create_all_corpus_train_pipeline(TIME_SLICE_SIZE)

In [ ]:

#g = corpus_training_data_generator('gutenberg',TIME_SLICE_SIZE, BATCH_SIZE, shift=False)
model.fit(X, Y, validation_split=0.2, callbacks=[tensor_board], batch_size=512, epochs=100)

Train on 273659 samples, validate on 68415 samples
Epoch 1/100
273659/273659 [==============================] - 137s 500us/step - loss: 0.0681 - acc: 0.9800 - val_loss: 0.0495 - val_acc: 0.9876
Epoch 2/100
273659/273659 [==============================] - 137s 500us/step - loss: 0.0668 - acc: 0.9803 - val_loss: 0.0474 - val_acc: 0.9882
Epoch 3/100
273659/273659 [==============================] - 137s 502us/step - loss: 0.0662 - acc: 0.9804 - val_loss: 0.0461 - val_acc: 0.9885
Epoch 4/100
273659/273659 [==============================] - 138s 503us/step - loss: 0.0658 - acc: 0.9805 - val_loss: 0.0461 - val_acc: 0.9887
Epoch 5/100
273659/273659 [==============================] - 139s 507us/step - loss: 0.0654 - acc: 0.9805 - val_loss: 0.0477 - val_acc: 0.9883
Epoch 6/100
273659/273659 [==============================] - 135s 494us/step - loss: 0.0650 - acc: 0.9806 - val_loss: 0.0479 - val_acc: 0.9880
Epoch 7/100
273659/273659 [==============================] - 134s 491us/step - loss: 0.0647

In [21]:
original_sentence = "his name was Michael Smith, he lived in Newcastle.".lower()
test_sentence = pad([original_sentence], TIME_SLICE_SIZE)
test_sentence = encode_each_sentence(test_sentence, input_feature_to_int_map)

mapped_sentence = np.asarray(test_sentence)
predicted_result = model.predict_classes(mapped_sentence)[0]
print(predicted_result)
predicted_result = list(zip(pad([original_sentence], TIME_SLICE_SIZE)[0], predicted_result.tolist()))

def true_case(letter, label):
    if letter == 0:
        return ''
    if(label == 2):
        return letter.lower()
    if (label == 3):
        return letter.upper()
    if (label == 0):
        return ''
    return letter

predicted_result = [true_case(letter, label) for letter, label in predicted_result]
''.join(predicted_result) 



[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]


'his name was michael smith, he lived in newcastle.'

In [ ]:
model.save("big_lstm.h5")

In [ ]:

for x in X[0]:
    print(x )

In [ ]:
Y